Zadanie 2
Wykorzystując przykłady zaprezentowane w labie wykonaj na danych people (możesz zmniejszyć lub zwiększyć ich wolumen w zależności od potrzeb) operację z użyciem Dask bag, która polegać będzie na przetworzeniu wszystkich plików i zapisaniu do plików o nazwie expired_{partition}.json rekordów, których ważność karty kredytowej wygasła (jest to wartość w formacie miesiąc/rok). Zapisując ustaw finalną liczbę plików na 10 jeżeli była inna. Możesz to zrobić poprzez zmianę ilość partycji dask bag (patrz link do API na początku laba).

Zadanie 3
Wybierając z danych people dane tylko osób dorosłych (zaprezentowane w przykładach w tym labie) przechowaj je w obiekcie typu bag, a następnie zapisz je do dask dataframe za pomocą metody to_dataframe (pamiętaj o tym jaka jest aktualna struktura pojedynczego rekordu), a następnie zapisz do jednego pliku w formacie parquet.

In [23]:
from dask.distributed import Client
import dask.dataframe as dd
import dask.bag as db
import json
import datetime
import dask.array as da


In [24]:
client = Client(n_workers=4)
array_size = (10000, 10000)
large_array = da.random.random(array_size, chunks=(1000, 1000))

result = large_array.mean().compute()

print("Mean of the large array:", result)

client.close()

C:\Users\dawid\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57288 instead
  warnings.warn(
C:\Users\dawid\anaconda3\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.68s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Mean of the large array: 0.49998347799981696


In [27]:
client = Client()

log_path = "Apache_2k.log"
logs = db.read_text(log_path)

error_logs = logs.filter(lambda line: "error" in line.lower())

error_lines = error_logs.compute()
for line in error_lines:
    print(line)

client.close()

C:\Users\dawid\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57479 instead
  warnings.warn(
C:\Users\dawid\anaconda3\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (2.42s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


[Sun Dec 04 04:47:44 2005] [error] mod_jk child workerEnv in error state 6

[Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6

[Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6

[Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6

[Sun Dec 04 04:51:55 2005] [error] mod_jk child workerEnv in error state 6

[Sun Dec 04 04:52:15 2005] [error] mod_jk child workerEnv in error state 6

[Sun Dec 04 04:52:15 2005] [error] mod_jk child workerEnv in error state 7

[Sun Dec 04 04:52:15 2005] [error] mod_jk child workerEnv in error state 7

[Sun Dec 04 04:52:52 2005] [error] mod_jk child workerEnv in error state 7

[Sun Dec 04 04:52:52 2005] [error] mod_jk child workerEnv in error state 6

[Sun Dec 04 04:53:16 2005] [error] mod_jk child workerEnv in error state 7

[Sun Dec 04 04:53:16 2005] [error] mod_jk child workerEnv in error state 6

[Sun Dec 04 04:53:54 2005] [error] mod_jk child workerEnv in error state 7

[Sun Dec 04 

In [17]:
client = Client(n_workers=4)
people_data = [
    {"name": "Jan Kowalski", "credit_card_expiry": "10/23"},
    {"name": "Anna Nowak", "credit_card_expiry": "05/21"},
    {"name": "John Smith", "credit_card_expiry": "07/24"},
    {"name": "Beny Alien", "credit_card_expiry": "02/20"}
]

bag = db.from_sequence(people_data)
def filter_expired(record):
    expiry = datetime.datetime.strptime(record['credit_card_expiry'], "%m/%y")
    return expiry < datetime.datetime.now()

expired_records = bag.filter(filter_expired).map(json.dumps)
expired_records.repartition(10).to_textfiles("expired_*.json")
client.close()

C:\Users\dawid\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56988 instead
  warnings.warn(


In [21]:
client.restart()
client = Client()

people_data = [{
    "name": "Alice", "age": 25}, 
    {"name": "Bob", "age": 17},
    {"name": "Jan", "age": 33},
    {"name": "Joan", "age": 18},
]

bag = db.from_sequence(people_data)

adults = bag.filter(lambda x: x['age'] >= 18)

adults_df = adults.to_dataframe()

adults_df.to_parquet("adults.parquet", write_metadata_file=True)

client.close()

2024-11-13 13:53:40,722 - distributed.nanny - WARNING - Restarting worker
2024-11-13 13:53:40,731 - distributed.nanny - WARNING - Restarting worker
2024-11-13 13:53:40,732 - distributed.nanny - WARNING - Restarting worker
2024-11-13 13:53:40,735 - distributed.nanny - WARNING - Restarting worker
C:\Users\dawid\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57224 instead
  warnings.warn(
C:\Users\dawid\anaconda3\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.36s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
2024-11-13 13:53:49,021 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "C:\Users\dawid\anaconda3\Lib\site-packages\distri

CancelledError: ('metadata-to-parquet-bb6fc185f85b3824fbaa9ded30899d00', 0)